In [1]:
import os 
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnableMap
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.agents.agent_toolkits import GmailToolkit
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.document_loaders.excel import UnstructuredExcelLoader

## 1. Create a model

In [8]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
gmail_toolkit = GmailToolkit()
llm=ChatOpenAI(temperature=0.0,model="gpt-3.5-turbo-0125")


### 2. Load the documents and split it

In [3]:
# Load the document
file_path = "info.xlsx"
loader = UnstructuredExcelLoader(file_path)
docs = loader.load()
# Split the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
splits = text_splitter.split_documents(docs)

# embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# To database

db = Chroma.from_documents(documents= splits, embedding = embeddings)
retreiver = db.as_retriever()

/Volumes/Ajeet/EmailGenerator/EmailGen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Create embeddings and retriever

### 4. Retrieving mails from ajeetacharya02@gmail.comm

In [9]:
output = {"emails":{
    "sender": "john.doe@example.com",
    "subject": "delivery charge?",
    "content": "What is the delivery charge?",
  }
}
type(output)



prompt ="""You have access to an external system that can check email. Please use it to see if I have received a any new email in the last one hour. If there are any, filter out non-essential ones such as newsletters, Blogs, promotional content and notifications. If i have not received any new mail then STOP the SEARCHUse your expertise in email content analysis to distinguish emails from the rest, pay attention to the sender and avoid invalid emails. ALWAYS REMEMBER,
    THE OUTPUT SHOULD BE RETURNED as a DICT not as a string  without double inverted commas in the output in the form given inside three backticks containing the sender's email address,the subject line of each email and the contents in the email,  separated by commas.
    
    Output: ```{"emails":{
                  "sender": "john.doe@example.com",
                  "subject": "delivery charge?",
                  "content": "What is the delivery charge?",
                }
              }   ```
    """

In [16]:
def check_for_mail():
    prompt ="""You have access to an external system that can check email. Please use it to see if I have received a any new email in the last one hour. If there are any, filter out non-essential ones such as newsletters, Blogs, promotional content and notifications. If i have not received any new mail then STOP the SEARCHUse your expertise in email content analysis to distinguish emails from the rest, pay attention to the sender and avoid invalid emails. ALWAYS REMEMBER,
    THE OUTPUT SHOULD BE RETURNED as a DICT not as a string  without double inverted commas in the output in the form given inside three backticks containing the sender's email address,the subject line of each email and the contents in the email,  separated by commas.
    
    Output: ```{"emails":{
                  "sender": "john.doe@example.com",
                  "subject": "delivery charge?",
                  "content": "What is the delivery charge?",
                }
              }   ```
    """

    agent = initialize_agent(agent= AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                            tools= gmail_toolkit.get_tools(), 
                            llm= llm, 
                            verbose = True, 
                            max_execution_time = 1600,
                            max_iterations = 1000)
    # prompt = PromptTemplate(template=template)
    response = agent.run(prompt)
    return response

In [17]:
# i = 0
# while i< 2:
#     print(f"-------------------------------Iteration number:{i}----------------------------")
response = check_for_mail()
#     i = i + 1



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-0125 in organization org-JjwTermCJqeO0oTlCgH2WTfa on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [15]:
response

{'emails': [{'sender': 'ajeetach97701@gmail.com',
   'subject': 'Hello',
   'content': 'What is the delivery cost?'}]}

In [6]:
prompt = """
You are a virtual assistant who answers users query. When a user asks a question  
you have to answer him based on the context given in three back ticks and make sure to provide each and every details.
context: ```{context}```
question is given in double back ticks : ``{question}``.
Provide a good answer that is related to context and DONOT EXCEED MORE THAN 30 WORDS.
If the question is out of context, do not provide the wrong answer just say i am sorry i don't have an answer to that question. Please contact info@palmminds.com or call to +977 9123123 for more info

"""

prompt = """

You are a virtual assistant who answers users query. When a user asks a question  
you have to answer him based on the context given in three back ticks and make sure to provide each and every details.
context: ```{context}```
question is given in double back ticks : ``{question}``.

If the question is out of context, do not provide the wrong answer just say i am sorry i don't have an ansawer to that question.

"""

In [7]:
template = ChatPromptTemplate.from_template(prompt)
output_parser = StrOutputParser()
chain = RunnableMap(
    {
        "context": lambda x:db.similarity_search(x['question'], k =1),
        "question": lambda x:x['question']
    }
)|template|llm|output_parser


In [8]:
# response

In [9]:
# email_data = response['emails']
# email_data

In [10]:
# to_send = email_data[1]

In [11]:
import os, smtplib
import email.message as em


def send_email(to_email, subject, message):
    smtp_server = os.getenv('SMTP_HOST')
    smtp_port = int(os.getenv('SMTP_PORT')) 
    sender_email = os.getenv('EMAIL_DEV')
    password = os.getenv('EMAIL_PASSWORD')

    msg = em.Message()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(message)

    smtp = smtplib.SMTP(smtp_server, smtp_port)
    smtp.starttls()
    smtp.login(sender_email, password)
    smtp.sendmail(msg['From'], [msg['To']], msg.as_string())
    smtp.quit()
    print("Successfully sent email to", to_email)


In [13]:
import time

def main():
    i = 0
    while i<= 2:
        time.sleep(10)
        print(f"-------------------------------Iteration number:{i}----------------------------")
        i += 1
        response = check_for_mail()  
        print(f"The response is:{response}")
        print(type(response))
        if response['emails']:
            email_data = response['emails']
            for email in email_data:
                result = chain.invoke({"question": email['content']})  
                print(email['sender'])
                send_email(to_email=email['sender'], subject=email['subject'], message=result) 
                print(result)
        else:
            print("No new email received")
        
if __name__ == "__main__":
    main()

-------------------------------Iteration number:0----------------------------


> Entering new AgentExecutor chain...
Thought: I need to search for any new emails in the last hour and filter out non-essential ones based on the sender and content.

Action:
```
{
  "action": "search_gmail",
  "action_input": {
    "query": "newer_than:1h",
    "resource": "messages",
    "max_results": 10
  }
}
```

Observation: [{'id': '18f04e5d63d0a6e8', 'threadId': '18f04e5d63d0a6e8', 'snippet': 'What is the delivery cost?', 'body': 'What is the delivery cost?\r\n', 'subject': 'Hello', 'sender': 'Ajeet Acharya <ajeetach97701@gmail.com>'}]
Thought:I will filter out non-essential emails and provide you with the relevant ones in the specified format.

Action:
```
{
  "action": "search_gmail",
  "action_input": {
    "query": "newer_than:1h",
    "resource": "messages",
    "max_results": 10
  }
}
```
Observation: [{'id': '18f04e5d63d0a6e8', 'threadId': '18f04e5d63d0a6e8', 'snippet': 'What is the delivery

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-0125 in organization org-JjwTermCJqeO0oTlCgH2WTfa on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}